In [1]:
! pip install -U wandb -i https://mirrors.aliyun.com/pypi/simple/
! pip install evaluate -i https://mirrors.aliyun.com/pypi/simple/
! pip install peft==0.12.0 -i https://mirrors.aliyun.com/pypi/simple/
! pip install -U bitsandbytes -i https://mirrors.aliyun.com/pypi/simple/
! pip install sacrebleu -i https://mirrors.aliyun.com/pypi/simple/
! pip install -U transformers -i https://mirrors.aliyun.com/pypi/simple/
! pip install datasets -i https://mirrors.aliyun.com/pypi/simple/
! pip install rouge_score -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import os
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, \
    Seq2SeqTrainer
import evaluate
import numpy as np
import wandb 
from  datasets import Dataset

dataset = load_dataset("skyfuryLH/semeval2025") 

print(dataset)
# 32962
train_val = dataset['train'].train_test_split(test_size=0.1)
train_data = train_val['train']
print(train_data)
# 29665
val_data = train_val['test']
print(val_data)
# 3297

DatasetDict({
    train: Dataset({
        features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
        num_rows: 32962
    })
})
Dataset({
    features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
    num_rows: 29665
})
Dataset({
    features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
    num_rows: 3297
})


In [3]:
if __name__=="__main__":
    os.environ['WANDB_API_KEY'] = "a464ce6c3b972e3e7090ac20839b9a1daac1b608"
    os.environ["WANDB_PROJECT"] = "T5-large-on-full-with-rouge-bleu-max_len=40"
    wandb.init()
    
    # 尝试移除无用列
    column_name = train_data.column_names

    model_id = "google-t5/t5-large"

    tokenizer = AutoTokenizer.from_pretrained(model_id)

#     prefix = '''Given a input sentence from the source language (en) that contains named entities, your task is to accurately translate the named entities in the input sentence to the target language {}: '''
    prefix = "translate English to {}: "
    # metric = evaluate.load("sacrebleu")

    bleuM = evaluate.load("sacrebleu")
    rougeM = evaluate.load("rouge")

    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    # 使用data_collator将数据集中的多个样本合并成一个batch，对齐输入和标签的长度
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)
    def preprocess_function(examples):
#         '''
#         在需要处理一个批次样本时，可以通过设置 batched=True，让 map 传入一组样本的批次。
#         此时，自定义函数的参数会变成一个包含每列字段的字典，字段的值为一个列表，代表该批次中所有样本在对应列下的数据。
#         这样，用户可以灵活选择是否逐条或批量处理数据。
#         :param examples:
#         :return:
#         '''
#         inputs = [prefix.format(target_lcoale)+example for example,target_lcoale in zip(examples['source'],examples['target_locale'])]
        translate_dict={
            "ar":"Arabic",
            "de":"German",
            "es":"Spanish",
            "fr":"French",
            "it":"Italian",
            "ja":"Japanese",
        }
        inputs = [prefix.format(translate_dict[target_lcoale])+example for example,target_lcoale in zip(examples['source'],examples['target_locale'])]
        
        
        targets = examples['target']
        
        
#         model_inputs = tokenizer(inputs,text_target=targets,max_length=50,truncation=True,padding=True,return_tensors="pt")

        model_inputs = tokenizer(inputs, max_length=40, truncation=True, padding="max_length", return_tensors='pt')
        labels = tokenizer(targets, max_length=40, truncation=True, padding="max_length",return_tensors='pt')

        # 将目标文本的填充 token 设置为 -100，以便在计算损失时忽略
        labels["input_ids"] = [
            [l if l != tokenizer.pad_token_id else -100 for l in label] for label in labels["input_ids"]
        ]
        # 将编码后的目标设置为模型的标签
        model_inputs["labels"] = labels["input_ids"]
    
       
        return model_inputs

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skyfurynowonline (skyfurynowonline-yunnan-university). Use `wandb login --relogin` to force relogin


In [4]:
print(train_data)
# 29665
print(train_data[0])

Dataset({
    features: ['id', 'source_locale', 'target_locale', 'source', 'target', 'entities', 'from'],
    num_rows: 29665
})
{'id': 'c8003610', 'source_locale': 'en', 'target_locale': 'ja', 'source': 'Did Donald Trump win the 2020 US Presidential Election?', 'target': 'ドナルド・トランプは、2020年のアメリカ大統領選挙で勝ちましたか？', 'entities': ['Q22686'], 'from': 'mintaka'}


In [5]:
    tokenized_train = train_data.map(preprocess_function,batched=True,remove_columns=column_name)
    tokenized_val = val_data.map(preprocess_function,batched=True,remove_columns=column_name)
    # 使用一部分进行验证
#     tokenized_val_slice = Dataset.from_dict(tokenized_val[0:1000])

Map:   0%|          | 0/29665 [00:00<?, ? examples/s]

Map:   0%|          | 0/3297 [00:00<?, ? examples/s]

In [6]:
    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]
        return preds, labels


    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        
        if isinstance(preds, tuple):
            preds = preds[0]
            
#         print(preds.shape)
#         batchsize,seq_len,vocab_size
#         (100, 64, 32128)
#         input()
        
        # 将 predictions 从概率转为 token IDs ?
        preds = np.argmax(preds,axis=-1) # (batch_size, seq_len)
        
        # 需要额外处理label吗？
        # transformers 中，填充位置通常被标记为 -100，需要将这些位置替换为 tokenizer.pad_token_id，避免在解码时生成错误文本
        
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        
        result = rougeM.compute(predictions=decoded_preds,references=decoded_labels)
        reuslt_bleu = bleuM.compute(predictions=decoded_preds,references=decoded_labels)
        result['bleu'] = reuslt_bleu['score']

        # result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        # result = {"bleu": result["score"]}
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        
        # result = {k: round(v, 4) for k, v in result.items()}
        return result

In [ ]:
    # 定义训练的超参数
    training_args = Seq2SeqTrainingArguments(
        output_dir='./semeval-T5large-with-rouge-max_len=40',  # output directory
        save_total_limit=3,       # 只保留最近的 3 个检查点
#         report_to="wandb",
        num_train_epochs=4,  # total number of training epochs
        per_device_train_batch_size=8,  # batch size per device during training
        per_device_eval_batch_size=16,  # batch size for evaluation
        warmup_steps=500,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        
        # 频繁的日志记录可能导致内存增加，特别是当模型需要频繁评估时。
        logging_steps=500,
        learning_rate=2e-5,  # Set learning rate
        evaluation_strategy="steps",
        
        eval_accumulation_steps=200,
        
        # 使用T5-base时遇到loss为NAN的问题，或许尝试不使用gradient_checkpointing
        # 尝试禁用gradient_checkpointing和FP16，跑出了结果
        
        # 过多裁剪会导致不能学习到有用的知识
#         max_grad_norm=1.0,  # 最大梯度裁剪，防止梯度爆炸
        
        save_strategy="epoch",
        
        gradient_checkpointing=True,
#         fp16=True, # 开启混合精度
        # gradient_accumulation_steps=8,  # 累积 8 个小批次
    )

    trainer = Seq2SeqTrainer(
        model = model,
        args = training_args,
        train_dataset = tokenized_train,
        eval_dataset = tokenized_val,
#         eval_dataset=tokenized_val_slice,
        tokenizer = tokenizer,
        data_collator = data_collator,
        compute_metrics = compute_metrics,
    )

    trainer.train()
    wandb.finish()
    trainer.save_model("./my_model_final-T5large-with-rouge-max_len=40")

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1496/2108896380.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len
500,0.969200,0.623575,0.338676,0.224028,0.336449,0.335953,25.065122,40.000000
1000,0.693800,0.550304,0.348297,0.239206,0.346226,0.345766,26.197366,40.000000
1500,0.633600,0.522904,0.352362,0.244056,0.350575,0.350394,25.477002,40.000000
2000,0.592600,0.496046,0.357942,0.252437,0.355991,0.356024,26.500190,40.000000
2500,0.572700,0.481451,0.367710,0.258903,0.366303,0.365958,27.713905,40.000000
3000,0.569900,0.466304,0.369425,0.261000,0.367712,0.367595,28.148641,40.000000
3500,0.551100,0.459421,0.369302,0.261951,0.367611,0.367304,27.232767,40.000000
4000,0.515800,0.451642,0.368798,0.267424,0.367221,0.366853,27.789165,40.000000
4500,0.485100,0.444796,0.375289,0.269894,0.373685,0.373590,27.586324,40.000000
5000,0.490300,0.439892,0.382050,0.271533,0.380487,0.380346,28.015759,40.000000


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("./my_model_final-T5large-with-rouge-max_len=40").to("cuda")
tokenizerSelf = AutoTokenizer.from_pretrained("./my_model_final-T5large-with-rouge-max_len=40")
text = "translate English to French: Hello, how are you ? "
inputs = tokenizerSelf(text,return_tensors="pt").to("cuda")
outputs = model.generate(inputs['input_ids'],max_length=64,num_beams=5,early_stopping=True)

decoded_output = tokenizerSelf.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")
text = "translate English to French: Hello, how are you ? "
inputs = tokenizer(text,return_tensors="pt").to("cuda")
outputs = model.generate(inputs['input_ids'],max_length=64,num_beams=5,early_stopping=True)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)